**This jupyter notebook demonstrates how to:**

1. Access Google drive files and folders using Google API;
2. Read and write files using api.

This resource was usefull to transfer files from/to Google Drive to/from Google Cloud Machine (could also be a local machine)

Post used to develope this notebook:

https://medium.com/@umdfirecoml/a-step-by-step-guide-on-how-to-download-your-google-drive-data-to-your-jupyter-notebook-using-the-52f4ce63c66c

API created tokens created with:

https://console.cloud.google.com/apis/credentials

In [ ]:
#install lib that is found at conda enviroment by default
!pip install oauth2client

In [ ]:
from googleapiclient import discovery
from httplib2 import Http
import oauth2client
from oauth2client import file, client, tools
import httplib2
import io
import os.path

from googleapiclient.http import MediaIoBaseDownload, MediaFileUpload



obj = lambda: None
lmao = {"auth_host_name":'localhost', 'noauth_local_webserver':'store_true', 'auth_host_port':[8080, 8090,5000], 'logging_level':'ERROR'}
for k, v in lmao.items():
    setattr(obj, k, v)
    
# authorization boilerplate code
SCOPES = 'https://www.googleapis.com/auth/drive'

#both token.json and client_id.json are obtained wih https://console.cloud.google.com/apis/credentials
store = file.Storage('token.json')
creds = store.get()

# The following will give you a link if token.json does not exist, the link allows the user to give this app permission
if not creds or creds.invalid:
    flow = client.flow_from_clientsecrets('client_id.json', SCOPES)
    creds = tools.run_flow(flow, store, obj)

In [ ]:
#build credentials
DRIVE = discovery.build('drive', 'v3', http=creds.authorize(Http()))

In [ ]:
def get_folder(name):
   """function to return folder from name:

    Args:
        name (str): folder name
        
    Returns:
        folder object
    """  
    results = DRIVE.files().list(
        pageSize=10,
        q=("name = '{0}'".format(name) +
           " and mimeType = 'application/vnd.google-apps.folder'"),
        corpora="user",
        fields="nextPageToken, files(id, name, webContentLink, " +
               "createdTime, modifiedTime)").execute()
    items = results.get('files', [])
    if not items:
        return None
    return items[0]

def get_folder_contents(folder_id):
   """function to return folders contents:

    Args:
        folder_id (): folder id returned by api
        
    Returns:
        folder contents
    """
    results = DRIVE.files().list(
        q=("'{0}' in parents".format(folder_id)),
        corpora="user",
        fields="nextPageToken, files(id, name, webContentLink, " +
               "createdTime, modifiedTime, size)").execute()
    items = results.get('files', [])
    if not items:
        print('No files found.')
    return items

In [ ]:
#get datasets folder
folder = get_folder('datasets')
#list files from folder
folder_items = get_folder_contents(folder['id'])

#download files from remote folder
for file in folder_items:
    file_name = file['name']
    #if file already exists check next file
    if os.path.exists(file_name):
        local_file_size = os.path.getsize(file_name)
        remote_file_size = file['size']
        #check if sizes are different
        if int(local_file_size) != int(remote_file_size):
            print(f"Arquivo {file_name} com tamanho diferente - remoto: {remote_file_size} local: {local_file_size}")
       
        continue

    #get the media request to perform in chunks.
    request = DRIVE.files().get_media(fileId=file['id'])

    #creates file handle to receive the content of the remote file
    fh = io.FileIO(file['name'], mode='w')

    downloader = MediaIoBaseDownload(fh, request)
    done = False
    while done is False:
        status, done = downloader.next_chunk()
        print (f" Download - {file_name} {int(status.progress() * 100)}%.")
    
    
    
    

In [ ]:
#upload file into folder
file_name = "file.csv"
folder_id ='<folder url from google drive>' #folder url 
file_metadata = {
    'name': file_name,
    'parents': [folder_id]

}
media = MediaFileUpload(file_name,
                        mimetype='text/csv',
                        resumable=True)
file = DRIVE.files().create(body=file_metadata,
                            media_body=media,
                            fields='id').execute()

print ('File {} ID: {}'.format(file_name, file.get('id')))